# LAB1 Wyszukiwanie wzorca w tekście - Jakub Janicki

## 0. Zaimplementuj algorytmy wyszukiwania wzorców:

### 0.1 Naiwny

In [9]:
import time

In [10]:
def naive(text, pattern):
    result = []
    for i in range(len(text) - len(pattern) + 1):
        if text[i:i + len(pattern)] == pattern:
            result.append(i)
    return result

### 0.2 Automat skończony

In [11]:
def finite_automata(text, pattern, delta = None):
    q = 0
    result = []
    if delta is None:
        delta = transition_table(pattern)
    for i, a in enumerate(text):
        q = delta[q].get(a, 0)
        if q == len(delta) - 1:
            result.append(i + 1 - q)
    return result

def transition_table(pattern):
    delta = []
    columns = set(pattern)

    for q in range(len(pattern) + 1):
        delta.append({})
        for a in columns:
            k = min(len(pattern), q + 1)        
            while not (pattern[:q] + a).endswith(pattern[:k]):
                k = k-1
            delta[q][a] = k
    return delta

In [12]:
transition_table("aaaab")

[{'b': 0, 'a': 1},
 {'b': 0, 'a': 2},
 {'b': 0, 'a': 3},
 {'b': 0, 'a': 4},
 {'b': 5, 'a': 4},
 {'b': 0, 'a': 1}]

### 0.3 KMP

In [13]:
def kmp(text, pattern, T = None):
    result = []
    q = 0
    if T is None:
        T = prefix_function(pattern)
    for i, a in enumerate(text):
        while q > 0 and pattern[q] != a:
            q = T[q - 1]
        if pattern[q] == a:
            q = q + 1
        if q == len(pattern):
            result.append(i + 1 - q)
            q = T[q - 1]
    return result


def prefix_function(pattern):
    pi = [0]
    q = 0
    for a in pattern[1:]:
        while q > 0 and pattern[q] != a:
            q = pi[q - 1]
        if a == pattern[q]:
            q = q + 1
        pi.append(q)
    return pi

In [14]:
prefix_function("aaaaab")

[0, 1, 2, 3, 4, 0]

## 1. Zaimplementuj testy porównujące szybkość działania wyżej wymienionych algorytmów.


In [15]:
def timer(func, text, pattern):
    start = time.time()
    func(text, pattern)
    end = time.time()
    return int(round((end - start)*(10**6),0))


def time_comparator(text, pattern):
    if not finite_automata(text, pattern) == naive(text, pattern) == kmp(text, pattern):
        result = False
    else:
        result = [{"Naive      ": timer(naive, text, pattern)},
                  {"Automata   ": timer(finite_automata, text, pattern)},
                  {"KMP        ": timer(kmp, text, pattern)}]

    print(*result, sep="\n")

## 2. Znajdź wszystkie wystąpienia wzorca "art" w załączonej ustawie, za pomocą każdego algorytmu.

In [16]:
f = open("assets/1997_714.txt", encoding='utf8')
art = f.read()


if not finite_automata(art,"art") == naive(art,"art") == kmp(art,"art"):
    result = "Roznia sie "
else:
    result = kmp(art, "art")
    print("Znaleziono " + str(len(result)) + " powtórzeń" )
    
print(result)

Znaleziono 273 powtórzeń
[1156, 1505, 4692, 4734, 4879, 5082, 5148, 5949, 6039, 7266, 7511, 7781, 8044, 8299, 9104, 9959, 10022, 10224, 11122, 11207, 11618, 13194, 15284, 15358, 16092, 16261, 16406, 16547, 16616, 16840, 16856, 23637, 24061, 24152, 24586, 24683, 24780, 24931, 25530, 25689, 27001, 27288, 27479, 27542, 27592, 27857, 28373, 28558, 28766, 30964, 31021, 31096, 31362, 31811, 32609, 32968, 33053, 33268, 33595, 34651, 34737, 35511, 36155, 37143, 37543, 38451, 38595, 39056, 39210, 39436, 39568, 39980, 41152, 41829, 42028, 42198, 42371, 42504, 42718, 42896, 42941, 43447, 43555, 43787, 44590, 44653, 44953, 45010, 45293, 45401, 47319, 47422, 48785, 48820, 48906, 49052, 49259, 49316, 49488, 49559, 49915, 49979, 50102, 50160, 50702, 51050, 51179, 51966, 52071, 52272, 52552, 53008, 53032, 53211, 53788, 53931, 54078, 54137, 54770, 55075, 55279, 55465, 55807, 55991, 56827, 56911, 57164, 57549, 57800, 57932, 57989, 58280, 58378, 58874, 58966, 59395, 59523, 59949, 60296, 60549, 60794, 612

## 3. Porównaj szybkość działania algorytmów dla problemu z p. 2.

In [17]:
time_comparator(art,"art")

{'Naive      ': 94908}
{'Automata   ': 107714}
{'KMP        ': 85801}


## 4.Zaproponuj tekst oraz wzorzec, dla którego zmierzony czas działania algorytmów (uwzględniający tylko dopasowanie, bez pre-processingu) automatu skończonego oraz KMP będzie co najmniej 5-krotnie krótszy niż dla algorytmu naiwnego.

In [18]:
pattern = 'a'*100000
text = ('a'*99999 + 'b')*10

prefixTable = prefix_function(pattern)
transitionTable = transition_table(pattern)

def timer2(func,text,pattern, table):
    start = time.time()
    func(text, pattern, table)
    end = time.time()
    return int(round((end - start)*(10**6),0))

result = [{"Naive      ": timer(naive, text, pattern)},
          {"Automata   ": timer2(finite_automata, text, pattern, transitionTable)},
          {"KMP        ": timer2(kmp, text, pattern, prefixTable)}]

print(*result, sep="\n")

{'Naive      ': 8405430}
{'Automata   ': 439114}
{'KMP        ': 1006901}


## 5. Zaproponuj wzorzec, dla którego zmierzony czas obliczenia tablicy przejścia automatu skończonego będzie co najmniej 5-krotnie dłuższy, niż czas potrzebny na utworzenie funkcji przejścia w algorytmie KMP.

In [19]:
import string
pattern2 = "string.ascii_lowercase"
def timer3(func, pattern):
    start = time.time()
    func(pattern)
    end = time.time()
    return int(round((end - start)*(10**6),0))


result = [{"transition_table" : timer3(transition_table, pattern2)}, 
          {"prefix_function " : timer3(prefix_function, pattern2)}]

print(*result, sep="\n")

{'transition_table': 2993}
{'prefix_function ': 0}
